<p align="center" width="100%">
    <img width="66%" src="https://raw.githubusercontent.com/linukc/master_dlcourse/main/images/logo.png">
</p>

 # **[MIPT DL frameworks Autumn 2025](https://wiki.cogmodel.mipt.ru/s/mtai/doc/2025-nejrosetevye-frejmvorki-glubokogo-obucheniya-y8VYap3O9E). Class 2: Segmentation**

# Самостоятельная работа

На основе библиотеки https://github.com/qubvel-org/segmentation_models.pytorch (или любой другой, выбранной вами) реализовать файнтюнинг модели семантической/инстанс сегментации на основе предоставленный весов (ImageNet, ...). В качестве набора данных можно использовать любой датасет, домен которого отличается от предоставленных весов. Обязательно провести 3 эксперимента с вариациями https://smp.readthedocs.io/en/latest/losses.html и 2 эксперимента с https://smp.readthedocs.io/en/latest/encoders.html. LR - ReduceLROnPlateau с warmup.

В качестве отчета предоставляется репозиторий с кодом, README.md с общими комментариями и pdf лог обучения (wandb, cometml, ...). Обязательно наличие метрик сегмемнтации на валидационной/тестовой выборке до/после обучения (минимум IoU).

Вместо репозитория можно предоставить блокнот с таким же содержанием.

https://smp.readthedocs.io/en/latest/quickstart.html

In [ ]:
model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=3,                      # model output channels (number of classes in your dataset)
)

In [ ]:
from segmentation_models_pytorch.encoders import get_preprocessing_fn

preprocess_input = get_preprocessing_fn('resnet18', pretrained='imagenet')

В качестве датасета можно использовать библиотеку [datasets](https://huggingface.co/docs/datasets).


[Пример семантической сегментации](https://huggingface.co/docs/datasets/semantic_segmentation)


[Список с датасетами](https://huggingface.co/datasets?modality=modality:image&task_categories=task_categories:image-segmentation&sort=trending)

Иногда сегментация предоставляется в качестве полигонов, а не изображения. Здесь код, который переводит полигоны в формат H x W

In [ ]:
import numpy as np
import cv2

# Example: multiple polygons
poly_data = [
    [
        [356.5, 494, 379, 494.5, 379.5, 452, 357, 451.5, 356.5, 494]
    ],
    [
        [100, 100, 150, 100, 150, 150, 100, 150, 100, 100]
    ]
]

H, W = 600, 800
labels = [1, 2]

def from_poly_to_mask(poly_data, labels, height, width):
    mask = np.zeros((height, width), dtype=np.int32)  # use int mask for labels
    for poly, label in zip(poly_data, labels):
        coords = np.array(poly[0], dtype=np.float32).reshape(-1, 2)
        coords = np.round(coords).astype(np.int32).reshape((-1, 1, 2))
        cv2.fillPoly(mask, [coords], label)
    return mask

# mask is HxW with multiple filled polygons
mask = from_poly_to_mask(poly_data, labels, H, W)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [ ]:
for images, gt_masks in dataloader:

    predicted_mask = model(image)
    loss = loss_fn(predicted_mask, gt_masks)

    loss.backward()
    optimizer.step()

NameError: name 'dataloader' is not defined

Оценка:

1 балл - задание полностью соответствует критериям